# Ensembling: Blending and Stacking

In this notebook, we implement stacking of machine learning models. Stacking several uncorrelated models is known to generalize better than individual models. Stacking mainly requires good cross-validation strategy between levels of prediction. In particular, we will demostrate that maintaining the same cross-validation folds between levels minimizes overfitting.

In [1]:
import pandas as pd
import numpy as np
import time
import joblib
import random
from joblib import Parallel, delayed
from scipy.optimize import minimize
from xgboost import XGBClassifier

from sklearn import model_selection, linear_model, metrics, decomposition, ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone

from tqdm.notebook import tqdm as tqdm
from functools import partial, reduce
from typing import List
import warnings
warnings.simplefilter(action='ignore')


NUM_FOLDS = 5

## Dataset

We do not really care too much about the dataset. The dataset used here is particularly nice. No issues. Idea is that we have text data in the form of a movie review, along with its sentiment classification. We will build a **sentiment classifier** using an ensemble of three models.

In [2]:
df = pd.read_csv('../input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/labeledTrainData.tsv', sep='\t')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


### Train and test split

In [3]:
df_train, df_test = model_selection.train_test_split(df, test_size=0.20)
print(df_train.shape, df_test.shape)

(20000, 3) (5000, 3)


### Cross-validation folds

Here we create cross-validation folds. Very important for evaluating models, and creating Level 1 features that are not overfitted.

In [4]:
df_train.loc[:, 'kfold'] = -1 
df_train = df_train.sample(frac=1.0).reset_index(drop=True)
y = df_train['sentiment'].values

skf = model_selection.StratifiedKFold(n_splits=NUM_FOLDS)
for f, (t_, v_) in enumerate(skf.split(X=df_train, y=y)):
    df_train.loc[v_, "kfold"] = f

In [5]:
df_train.kfold.value_counts()

0    4000
1    4000
2    4000
3    4000
4    4000
Name: kfold, dtype: int64

## Stacking and Blending

We define a class that automates training and prediction of stacked models. Several models can be trained on the training set whose predict probabilities can be used as feature for a further metamodel called a **stacker**. Observe that this process can be iterated to several more levels. To avoid creating meta features that are overfitted to the train set, the meta features are generated by out-of-fold (OOF) training and prediction of the models on the features of the previous level. This requires defining cross-validation folds. The same cross-validation folds will be used to generate metafeatures at deeper levels. This will be justified later. 

After generating metafeatures, the models will be retrained on the whole training set (not just on train folds). This increases accuracy of prediction on the test set. Finally, prediction on the test set will simulate conditions when the model was trained &mdash; essentially the test set acts like an extra validation fold.


:::{note}
Alternatively, we could make predictions on the test dataset using each base model immediately after it gets fitted on each fold. In our case, this would generate test-set predictions for five of each base models. Then, we would average the predictions per model to generate our level 1 meta features.

One benefit to this is that it’s less time consuming than the first approach (since we don’t have to retrain each model on the full training dataset). It also helps that our train meta features and test meta features should follow a similar distribution. However, the test meta features are likely more accurate in the first approach since each base model was trained on the full training dataset (as opposed to 80% of the training dataset, five times in the 2nd approach).
:::



### Implementation

In [6]:
class StackingClassifier:
    """Implements model stacking for classification."""
    
    def __init__(self, model_dict_list, verbose=True):
        """Initialize by passing a list of dictionaries of name-model pairs 
        for each level."""
        
        self.model_dict_list = model_dict_list
        self.cv_scores_ = {}
        self.metafeatures_ = None
        self.verbose = verbose
        
    def fit(self, df):
        """Fit classifier. This assumes `df` is a DataFrame with "id", "kfold", 
        "sentiment" (target) columns, followed by features columns."""
                
        # Iterating over all stacking levels
        df = df.copy()
        metafeatures = []
        for m in range(len(self.model_dict_list)):
            
            # Get models in current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Identify feature columns, i.e. preds of prev. layer
            if m == 0:
                feature_cols = ['review']
            else:
                prev_level_names = self.model_dict_list[m-1].keys()
                feature_cols = [f'{name}_{level-1}' for name in prev_level_names]
            
            # Iterate over models in the current layer
            for model_name in model_dict.keys():
                if self.verbose:
                    print(f'\nLevel {level} preds: {model_name}')
                self.cv_scores_[f'{model_name}_{level}'] = []
                model = model_dict[model_name]
                
                # Generate feature for next layer models from OOF preds
                oof_preds = []
                for j in range(df.kfold.nunique()):
                    oof_pred, oof_auc = self._oof_pred(df, feature_cols, model, 
                                                        model_name, fold=j, level=level)
                    oof_preds.append(oof_pred)
                    self.cv_scores_[f'{model_name}_{level}'].append(oof_auc)
                
                pred = pd.concat(oof_preds)
                df = df.merge(pred[['id', f'{model_name}_{level}']], on='id', how='left')   
                metafeatures.append(f'{model_name}_{level}')
        
                # Train models on entire feature columns for inference
                model.fit(df[feature_cols], df.sentiment.values)
        
        self.metafeatures_ = df[metafeatures]
        return self
        
    def predict_proba(self, test_df):
        """Return classification probabilities."""
        
        test_df = test_df.copy()
        
        # Iterate over layers to make predictions
        for m in range(len(self.model_dict_list)):
            
            # Get models for current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Get feature columns to use for prediction
            if m == 0:
                feature_cols = ['review']
            else:
                prev_names = self.model_dict_list[m-1].keys()
                feature_cols = [f"{model_name}_{level-1}" for model_name in prev_names]

            # Append predictions to test DataFrame
            for model_name in model_dict.keys():
                model = model_dict[model_name]
                pred = model.predict_proba(test_df[feature_cols])[:, 1] 
                test_df.loc[:, f"{model_name}_{level}"] = pred
                    
        # Return last predictions
        return np.c_[1 - pred, pred]
        
    def _oof_pred(self, df, feature_cols, model, model_name, fold, level):
        "Train on K-1 folds, predict on fold K. Return OOF predictions with IDs."

        # Get folds; include ID and target cols, and feature cols
        df_trn = df[df.kfold != fold][['id', 'sentiment']+feature_cols]
        df_oof = df[df.kfold == fold][['id', 'sentiment']+feature_cols]
        
        # Fit model.
        model.fit(df_trn[feature_cols], df_trn.sentiment.values)
        oof_pred = model.predict_proba(df_oof[feature_cols])[:, 1] 
        auc = metrics.roc_auc_score(df_oof.sentiment.values, oof_pred)
        if self.verbose:
            print(f"fold={fold}, auc={auc}")

        # Return OOF predictions with ids
        df_oof.loc[:, f"{model_name}_{level}"] = oof_pred
        return df_oof[["id", f"{model_name}_{level}"]], auc

### Blending

Let's start with a simple stacked model where we simply perform a weighted average of the prediction probabilities. This method is called **blending**. We will use three base models to generate probabilities. Hopefully these are uncorrelated:
1. Logistic Regression + TF-IDF
2. Logistic Regression + Count Vectorizer
3. Random Forest + TF-IDF + SVD

In [7]:
class ReviewColumnExtractor(BaseEstimator, ClassifierMixin):
    """Extract text column, e.g. letting X = df_train[['review']]
    as train dataset for TfidfVectorizer and CountVectorizer does
    not work as expected."""
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.review

Initialize base models:

In [8]:
lr = make_pipeline(
    ReviewColumnExtractor(),
    TfidfVectorizer(max_features=1000),
    linear_model.LogisticRegression()
)

lr_cnt = make_pipeline(
    ReviewColumnExtractor(),
    CountVectorizer(),
    linear_model.LogisticRegression(solver='liblinear')
)

rf_svd = make_pipeline(
    ReviewColumnExtractor(),
    TfidfVectorizer(max_features=None),
    decomposition.TruncatedSVD(n_components=120),
    ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
)

Run training:

In [9]:
basemodels = {'lr': lr, 'lr_cnt': lr_cnt, 'rf_svd': rf_svd}
stack = StackingClassifier([basemodels])
stack.fit(df_train)


Level 1 preds: lr
fold=0, auc=0.935408483852121
fold=1, auc=0.9348064837016209
fold=2, auc=0.9421081921081921
fold=3, auc=0.9339369339369339
fold=4, auc=0.9282464282464282

Level 1 preds: lr_cnt
fold=0, auc=0.9432962358240589
fold=1, auc=0.9477042369260591
fold=2, auc=0.9484371984371985
fold=3, auc=0.945093695093695
fold=4, auc=0.9360944360944361

Level 1 preds: rf_svd
fold=0, auc=0.8857182214295554
fold=1, auc=0.8764332191083047
fold=2, auc=0.886909636909637
fold=3, auc=0.8699118699118699
fold=4, auc=0.8746972496972496


Check if basemodels are uncorrelated:

In [10]:
stack.metafeatures_.corr()

,lr_1,lr_cnt_1,rf_svd_1
lr_1,1.000000,0.888748,0.831120
lr_cnt_1,0.888748,1.000000,0.725899
rf_svd_1,0.831120,0.725899,1.000000


The model saves learned probabilistic features:

In [11]:
stack.metafeatures_.head()

,lr_1,lr_cnt_1,rf_svd_1
0,0.830411,0.999716,0.69
1,0.649919,0.999999,0.53
2,0.865389,0.952598,0.63
3,0.209188,0.005921,0.29
4,0.815459,0.996732,0.38


We can also check scores of the base models on each validation fold. This informs us of the stability of the folds and the cross-validation performance of the base models. 

In [12]:
pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]

,lr_1,lr_cnt_1,rf_svd_1
mean,0.934901,0.944125,0.878734
std,0.004935,0.004938,0.007332


Let's try to blend the probabilities using some hand-designed coefficients.

In [13]:
target = df_train.sentiment.values

# roc is scale invariant, so we dont bother dividing by total weights
avg_preds = (stack.metafeatures_ * [1, 1, 1]).sum(axis=1)
wtd_preds = (stack.metafeatures_ * [1, 3, 1]).sum(axis=1)
rank_avg_preds = (stack.metafeatures_.rank() * [1, 1, 1]).sum(axis=1)
rank_wtd_preds = (stack.metafeatures_.rank() * [1, 3, 1]).sum(axis=1)

# Calculate AUC over combined OOF preds
print(f"Train OOF-AUC (averaged):     ", metrics.roc_auc_score(target, avg_preds))
print(f"Train OOF-AUC (wtd. avg):     ", metrics.roc_auc_score(target, wtd_preds))
print(f"Train OOF-AUC (rank avg):     ", metrics.roc_auc_score(target, rank_avg_preds)) 
print(f"Train OOF-AUC (wtd. rank avg):", metrics.roc_auc_score(target, rank_wtd_preds))

Train OOF-AUC (averaged):      0.9477839565817322
Train OOF-AUC (wtd. avg):      0.9489263573128687
Train OOF-AUC (rank avg):      0.94323259366886
Train OOF-AUC (wtd. rank avg): 0.9490437473879982


Since these coefficients are hand-designed, we may want to devise a strategy for automatically finding the optimal coefficients for blending. This is accomplished by the folowing class.

In [14]:
class Blender(BaseEstimator, ClassifierMixin):
    """Implement blending that maximizes AUC score."""
    
    def __init__(self, rank=False, random_state=42):
        self.coef_ = None
        self.rank = rank
        self.random_state = random_state

    def fit(self, X, y):
        """Find optimal blending coefficients."""
        
        if self.rank:
            X = X.rank()

        self.coef_ = self._optimize_auc(X, y)
        return self

    def predict_proba(self, X):
        """Return blended probabilities for class 0 and class 1."""
        
        if self.rank:
            X = X.rank()
            
        pred = np.sum(X * self.coef_, axis=1)
        return np.c_[1 - pred, pred]

    def _auc(self, coef, X, y):
        """Calculate AUC of blended predict probas."""

        auc = metrics.roc_auc_score(y, np.sum(X * coef, axis=1))
        return -1.0 * auc # min -auc = max auc
    
    def _optimize_auc(self, X, y):
        """Maximize AUC as a bound-constrained optimization problem using Nelder-Mead 
        method with Dirichlet init. 
        
        Reference: 
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
        """
        partial_loss = partial(self._auc, X=X, y=y) 
        rng = np.random.RandomState(self.random_state)
        init_coef = rng.dirichlet(np.ones(X.shape[1]))
        return minimize(partial_loss, init_coef, 
                        method='Nelder-Mead', 
                        bounds=[(0, 1)]*X.shape[1])['x']

This implementation uses `partial` from `functools` and `minimize` from `scipy.optimize` to minimize the coefficients constained in $(0, 1).$ The initial values of the coefficient are drawn from a Dirichlet distribution $\operatorname{Dir}(\boldsymbol{\alpha})$ with $\boldsymbol{\alpha} = [1, 1, 1].$

In [15]:
target = df_train.sentiment.values

# Blended predictions
blender = Blender()
blender.fit(stack.metafeatures_, target)
combined_oof_preds = (stack.metafeatures_ * blender.coef_).sum(axis=1)

# Blended ranked predictions
blender_rk = Blender(rank=True)
blender_rk.fit(stack.metafeatures_, target)
combined_oof_rk_preds = (stack.metafeatures_.rank() * blender_rk.coef_).sum(axis=1)

print(f"Train OOF-AUC (Blended):    ", metrics.roc_auc_score(target, combined_oof_preds))
print(f"Train OOF-AUC (Blended rk.):", metrics.roc_auc_score(target, combined_oof_rk_preds))

Train OOF-AUC (Blended):     0.9493579175890672
Train OOF-AUC (Blended rk.): 0.9498664579145331


Note that Train OOF-AUC is not the same as train AUC. However, this should be a better approximation of the test AUC. Calculating the AUC on the entire out-of-fold predictions involves tracking the rows of the confusion matrix, which is the sum of the confusion matrix of each fold, over all thresholds. On the other hand, the latter approach tracks each confusion matrix separately, then averages the individual AUCs. The two should be similar to cross-validation scores if error is well-distributed between folds &mdash; and we are blending probabilities. [^ref]

[^ref]: For some reason OOF-AUC is bad when blending ranking models, e.g. linear regression, with usual classifiers, even after transforming predict probabilities to rank.

In [16]:
# Inference
test_target = df_test.sentiment.values
test_features = []
for model_name in basemodels.keys():
    test_features.append(basemodels[model_name].predict_proba(df_test)[:, 1])

test_pred = (pd.DataFrame(np.c_[test_features].T) * blender.coef_).sum(axis=1)
test_rk_pred = (pd.DataFrame(np.c_[test_features].T) * blender_rk.coef_).sum(axis=1)
print('Test AUC (Blended):    ', metrics.roc_auc_score(test_target, test_pred))
print('Test AUC (Blended rk.):', metrics.roc_auc_score(test_target, test_rk_pred))

Test AUC (Blended):     0.9478908264020622
Test AUC (Blended rk.): 0.9478913064069776


:::{tip}
Using blended **rank probabilities** is a good trick when optimizing AUC score. Here individual probabilities are replaced by their rank index. Recall that AUC only cares about the predict probability of a randomly chosen negative examples to be assigned lower predict proba than a randomly chosen positive example. Note that this only works for ensembles; for single models using rank probabilities does not affect AUC score.
:::

### XGB Metamodel

Blending can be easily generalized to more complex machine learning model that learns and predicts with the metafeatures using more complex algorithms. For example, we can use `XGBoostClassifier`.

In [17]:
basemodels = {'lr': lr, 'lr_cnt': lr_cnt, 'rf_svd': rf_svd}
metamodel = {'xgb': XGBClassifier(eval_metric="logloss", use_label_encoder=False)}
stack = StackingClassifier([basemodels, metamodel])
stack.fit(df_train)


Level 1 preds: lr
fold=0, auc=0.935408483852121
fold=1, auc=0.9348064837016209
fold=2, auc=0.9421081921081921
fold=3, auc=0.9339369339369339
fold=4, auc=0.9282464282464282

Level 1 preds: lr_cnt
fold=0, auc=0.9432962358240589
fold=1, auc=0.9477042369260591
fold=2, auc=0.9484371984371985
fold=3, auc=0.945093695093695
fold=4, auc=0.9360944360944361

Level 1 preds: rf_svd
fold=0, auc=0.8854135963533991
fold=1, auc=0.8771230942807736
fold=2, auc=0.882949882949883
fold=3, auc=0.8682824932824932
fold=4, auc=0.870939870939871

Level 2 preds: xgb
fold=0, auc=0.9450343612585903
fold=1, auc=0.9459216114804029
fold=2, auc=0.9506855756855757
fold=3, auc=0.9444283194283194
fold=4, auc=0.9393175643175643


In [18]:
y_train = df_train.sentiment.values
y_test = df_test.sentiment.values

print(f"Train AUC (XGB stack):", metrics.roc_auc_score(y_train, stack.predict_proba(df_train)[:, 1]))
print(f"Test AUC  (XGB stack):", metrics.roc_auc_score(y_test, stack.predict_proba(df_test)[:, 1]))

Train AUC (XGB stack): 0.9995912697384126
Test AUC  (XGB stack): 0.9438695052237334


In [19]:
pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]

,lr_1,lr_cnt_1,rf_svd_1,xgb_2
mean,0.934901,0.944125,0.876942,0.945077
std,0.004935,0.004938,0.007398,0.004055


Observe that cross-validated AUC scores is indicative of test performance. Meanwhile, train AUC is useless. A better estimate is the mean cross-validation AUC score. If we assume that each fold has the same error distribution, then this should approximate the test AUC which can be thought of as predicting on another fold. Indeed, the above results supports this.

### Conclusion

The above examples of building ensembles with **blending** or **stacking** (e.g. with XGBoost) show that stacked models significantly outperform single models.

## Experiment: CV Folds

Consider stacking three levels of models. Our current implementation does this by keeping the same cross-validation folds when training the level 2 models [^ref2]. It is unclear whether using same folds between levels 1 and 2 affect generalization error. To check this empirically, we compute CV scores with fold indices shuffled after each level. This simulates changing fold assignments for each training example betweel levels. We calculate train and test set AUCs, as well as CV scores.

[^ref2]: GM Abishek Thakur recommends keeping the same folds in [AAAMLP](https://github.com/abhishekkrthakur/approachingalmost/blob/master/AAAMLP.pdf).

In [20]:
class LinearRegressionClassifier(BaseEstimator, ClassifierMixin):
    """Linear regression for model-based AUC optimization.
    Note that we transform probabilities to rank probabilities!"""
    
    def __init__(self): 
        self.lr = linear_model.LinearRegression()
        
    def fit(self, X, y):
        self.lr.fit(pd.DataFrame(X).rank(), y)
        return self
        
    def predict_proba(self, X):
        return np.c_[[0]*len(X), self.lr.predict(pd.DataFrame(X).rank())]

Define models for stacking.

In [21]:
# Base models
level1 = {
    'lr': make_pipeline(
        ReviewColumnExtractor(),
        TfidfVectorizer(max_features=1000),
        linear_model.LogisticRegression(random_state=42)
    ), 
    
    'lr_cnt': make_pipeline(
        ReviewColumnExtractor(),
        CountVectorizer(),
        linear_model.LogisticRegression(solver='liblinear', 
                                        random_state=42)
    ), 
}

# Meta models
level2 = {
    'lr': linear_model.LogisticRegression(random_state=42),
    
    'linreg': make_pipeline(
        StandardScaler(), 
        LinearRegressionClassifier()
    ),
    
    'xgb': XGBClassifier(eval_metric="logloss", 
                         use_label_encoder=False, 
                         random_state=42)
}

# Blender head: rank true for linear regression
level3 = {'blender': Blender(rank=True, random_state=42)}

### Same Folds

Run the experiment many times.

In [22]:
NUM_RUNS = 5
same_train_auc = []
same_test_auc = []

for i in tqdm(range(NUM_RUNS)):
    
    # Run training of stack models
    stack = StackingClassifier([level1, level2, level3], verbose=False)
    stack.fit(df_train)
    
    # Record results
    train_score = metrics.roc_auc_score(df_train.sentiment, stack.predict_proba(df_train)[:, 1])
    test_score  = metrics.roc_auc_score(df_test.sentiment,  stack.predict_proba(df_test )[:, 1])
    same_train_auc.append(train_score)
    same_test_auc .append(test_score )

    if i == 0:
        cv_results  = pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]
    else:
        cv_results += pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]
        
# Convert to arrays
same_train_auc = np.array(same_train_auc)
same_test_auc  = np.array(same_test_auc )

  0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
experiment_results = {}
experiment_results['same'] = {
    'train_mean': same_train_auc.mean(), 
    'train_std':  same_train_auc.std(),
    'test_mean':  same_test_auc .mean(),
    'test_std':   same_test_auc .std(),
}

### Shuffled Folds

Now we train the same model except the folds are shuffled beyond training the level 1 models simulating the use of different cross-validation folds when training higher level models.

In [24]:
class StackingClassifierShuffledCV:
    """Implements model stacking for classification."""
    
    def __init__(self, model_dict_list, verbose=True):
        """Initialize by passing a list of dictionaries of name-model pairs 
        for each level."""
        
        self.model_dict_list = model_dict_list
        self.cv_scores_ = {}
        self.metafeatures_ = None
        self.verbose = verbose
        
    def fit(self, df):
        """Fit classifier. This assumes `df` is a DataFrame with "id", "kfold", 
        "sentiment" (target) columns, followed by features columns."""
                
        # Iterating over all stacking levels
        df = df.copy()
        metafeatures = []
        for m in range(len(self.model_dict_list)):
            
            # Get models in current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Identify feature columns, i.e. preds of prev. layer
            if m == 0:
                feature_cols = ['review']
            else:
                prev_level_names = self.model_dict_list[m-1].keys()
                feature_cols = [f'{name}_{level-1}' for name in prev_level_names]
                df['kfold'] = random.sample(df.kfold.tolist(), len(df)) # SHUFFLE FOLDS HERE <!>
            
            # Iterate over models in the current layer
            for model_name in model_dict.keys():
                if self.verbose:
                    print(f'\nLevel {level} preds: {model_name}')
                self.cv_scores_[f'{model_name}_{level}'] = []
                model = model_dict[model_name]
                
                # Generate feature for next layer models from OOF preds
                oof_preds = []
                for j in range(df.kfold.nunique()):
                    oof_pred, oof_auc = self._oof_pred(df, feature_cols, model, 
                                                        model_name, fold=j, level=level)
                    oof_preds.append(oof_pred)
                    self.cv_scores_[f'{model_name}_{level}'].append(oof_auc)
                
                pred = pd.concat(oof_preds)
                df = df.merge(pred[['id', f'{model_name}_{level}']], on='id', how='left')   
                metafeatures.append(f'{model_name}_{level}')
        
                # Train models on entire feature columns for inference
                model.fit(df[feature_cols], df.sentiment.values)
        
        self.metafeatures_ = df[metafeatures]
        return self
        
    def predict_proba(self, test_df):
        """Return classification probabilities."""
        
        test_df = test_df.copy()
        
        # Iterate over layers to make predictions
        for m in range(len(self.model_dict_list)):
            
            # Get models for current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Get feature columns to use for prediction
            if m == 0:
                feature_cols = ['review']
            else:
                prev_names = self.model_dict_list[m-1].keys()
                feature_cols = [f"{model_name}_{level-1}" for model_name in prev_names]

            # Append predictions to test DataFrame
            for model_name in model_dict.keys():
                model = model_dict[model_name]
                pred = model.predict_proba(test_df[feature_cols])[:, 1] 
                test_df.loc[:, f"{model_name}_{level}"] = pred
                    
        # Return last predictions
        return np.c_[1 - pred, pred]
        
    def _oof_pred(self, df, feature_cols, model, model_name, fold, level):
        "Train on K-1 folds, predict on fold K. Return OOF predictions with IDs."

        # Get folds; include ID and target cols, and feature cols
        df_trn = df[df.kfold != fold][['id', 'sentiment']+feature_cols]
        df_oof = df[df.kfold == fold][['id', 'sentiment']+feature_cols]
        
        # Fit model.
        model.fit(df_trn[feature_cols], df_trn.sentiment.values)
        oof_pred = model.predict_proba(df_oof[feature_cols])[:, 1] 
        auc = metrics.roc_auc_score(df_oof.sentiment.values, oof_pred)
        if self.verbose:
            print(f"fold={fold}, auc={auc}")

        # Return OOF predictions with ids
        df_oof.loc[:, f"{model_name}_{level}"] = oof_pred
        return df_oof[["id", f"{model_name}_{level}"]], auc

:::{danger}
The implementation of `StackingClassifier` has a side-effect: models inside the dictionaries are trained. Hence, if we train another model using the same model dictionaries (as we do here in defining `stack_shuffled`), then the models inside the dictionaries will be retrained using a different algorithm. This means calling `stack.predict_proba(df_test)` will yield **different results** before and after training `stack_shuffled`! As usual, the stateful approach is error prone. We can modify the `StackingClassifier` to instead save a list of model dictionaries that are *clones* of the models. This allows all state to be localized within the  stacked model.
:::

Start training the stacked model with shuffling of folds:

In [25]:
shuffled_train_auc = []
shuffled_test_auc  = []

for i in tqdm(range(NUM_RUNS)):
    
    # Run training of stack models
    stack = StackingClassifierShuffledCV([level1, level2, level3], verbose=False)
    stack.fit(df_train)
    
    # Record results
    train_score = metrics.roc_auc_score(df_train.sentiment, stack.predict_proba(df_train)[:, 1])
    test_score  = metrics.roc_auc_score(df_test.sentiment,  stack.predict_proba(df_test )[:, 1])
    shuffled_train_auc.append(train_score)
    shuffled_test_auc .append(test_score )

    if i == 0:
        shuffled_cv_results  = pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]
    else:
        shuffled_cv_results += pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]
        
# Convert to arrays
shuffled_train_auc = np.array(shuffled_train_auc)
shuffled_test_auc  = np.array(shuffled_test_auc )

  0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
experiment_results['shuffled'] = {
    'train_mean': shuffled_train_auc.mean(), 
    'train_std':  shuffled_train_auc.std(),
    'test_mean':  shuffled_test_auc .mean(),
    'test_std':   shuffled_test_auc .std(),
}

In [27]:
pd.DataFrame(experiment_results)

,same,shuffled
train_mean,9.924016e-01,0.992848
train_std,1.110223e-16,0.000429
test_mean,9.483152e-01,0.948315
test_std,0.000000e+00,0.000022


Observe that the train AUC increased while test score decreased when using different folds between layers which indicates overfitting. However, if we look at cv scores, we see that while the mean of scores is higher for same cross-validation folds, the spread of values is larger compared to the shuffled folds.

In [28]:
# Shuffled CV scores
shuffled_cv_results / NUM_RUNS

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,blender_3
mean,0.934901,0.944125,0.948279,0.949361,0.945230,0.949464
std,0.004935,0.004938,0.002985,0.002963,0.003097,0.002746


In [29]:
# Same CV scores
cv_results / NUM_RUNS

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,blender_3
mean,0.934901,0.944125,0.948298,0.949398,0.945144,0.949624
std,0.004935,0.004938,0.004481,0.004642,0.003903,0.004561


### Conclusion

Empirical results above suggest that we should use the **same folds** across levels of stacking to maximize score. The following theoretical example shows that, when using different folds, overfitting can happen due to the second stage model taking advantage of a certain relationship between ground truth and first stage predictions, without this structure generalizing well to the test set.

Consider a dataset $\{(x_1, t_1), (x_2, t_2) \ldots, (x_{10}, t_{10})\}$ with five folds such that the first fold is $F_1 = \{x_1, x_2\}$. Let $x_1 {\mapsto} y_1$ and $x_2 \mapsto y_2$ where the mapping is trained on $F_{\neg 1} = \{x_3, \ldots, x_{10}\}.$ We can think of modelling on $F_{\neg 1}$ as defining some rule or distribution that the points in $F_1$ are compared against. Suppose we reshuffle folds in the next level such that the first fold is $G_1 = \{y_1, y_{10}\}.$ Then, the model trained on $G_{\neg 1} = \{y_2, \ldots y_9\}$  overfits slightly since $y_2$ is modelled using the ground truths $(x_3, t_3), \ldots, (x_{9}, t_9).$ Note that this asymmetry does not apply to the other values $y_3, \ldots, y_9$ in $G_{\neg 1}.$ Keeping the same cross-validation scores allow all instances in the train fold are equivalent and prevents any such asymmetry from occurring. 

Theoretically (as mentioned in the [Kaggle Guide to Model Stacking](https://datasciblog.github.io/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/)), there is always some leakage if you train a second level model on the same training set, which you used to derive the first stage predictions. This is because you used the ground truth to get those first stage predictions, and now you take those predictions as input, and try to predict the same ground truth. However, this leakage doesn't seem to be significant in practice.

## Parallelizing Model Training


Generating features require training one model for each fold. This is very slow. Note that each training process is independent of the other (they only use static features from the previous level), so in principle can be easily parallelized. For this task, we parallelize only the training on cross-validation folds. During inference, parallelizing results in worse times, likely due to overhead. 

We implement parallelizing training on CV folds using `joblib.Parallel`. Some remarks:

* Setting `backend='loky'` is important. Locally (Mojave 10.14.6), setting `backend='multiprocessing'` with an XGBoost classifier causes training to hang. In a Kaggle kernel, `multiprocessing` doesn't seem to work at all, even without using an XGBoost model. Using the `loky` backend seems to work consistently across platforms. 

+++

* Setting `nthread=1` for XGBClassifier decreases train trime from ~250s to ~100s with backend `loky` and `n_jobs=-1`. Note that the former time is way worse than sequential evaluation. 

+++

* Joblib pickles every object used inside `Parallel`. Best to use stateless objects. Careful about shared memory. Using `n_jobs=1` turns off parallel computing for debugging.

Note that we will start **cloning** models in the `model_dict_list` inside the `StackingCLassifierParallel` object to avoid leaking state outside the object instance. Results below show that there is significant speed up with parallelization using the `loky` backend. Consider this implementation the current stable version of our implementation of stacking.

In [30]:
class StackingClassifierParallel(BaseEstimator, ClassifierMixin):
    """Implements model stacking for classification."""
    
    def __init__(self, model_dict_list, n_jobs=1, backend='loky'):
        """Initialize by passing `model_dict` which is a list of dictionaries 
        of name-model pairs for each level. Models should have inter-level
        unique names."""
        
        self.model_dict_list = [
            { name: clone(model_dict[name]) for name in model_dict } 
            for model_dict in model_dict_list
        ]
        self.cv_scores_ = {}
        self.metafeatures_ = None
        self.n_jobs = n_jobs
        self.backend = backend
    
    def fit(self, df):
        """Fit classifier. Assumes `df` is a DataFrame with 'id', 'kfold', and 
        'sentiment' (target) columns, followed by features columns."""
        
        # Iterating over all stacking levels
        df = df.copy()
        metafeatures = []
        for m in tqdm(range(len(self.model_dict_list)), leave=False):
            
            # Get models in current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Identify feature columns, i.e. preds of prev. layer
            if m == 0:
                feature_cols = ['review']
            else:
                prev_level_names = self.model_dict_list[m-1].keys()
                feature_cols = [f'{name}_{level-1}' for name in prev_level_names]
            
            # Parallel context manager. Prevents discarding of workers for each model
            with Parallel(n_jobs=self.n_jobs, backend=self.backend, verbose=1) as parallel:
                
                # Iterate over models in the current layer
                for model_name in tqdm(model_dict.keys(), leave=False):
                    
                    # Generate feature for next layer models from OOF preds
                    # Cloning the model here releases the weights from prev. fit.
                    model = model_dict[model_name]
                    out = parallel(delayed(self._predict_fold)(
                            df, feature_cols, fold,
                            model_name, clone(model),
                            level
                        ) for fold in df.kfold.unique()
                    )

                    # Load all OOF predictions and AUCs
                    fold_preds, cv_scores = list(zip(*out))
                    
                    # Assign cv scores for model and append predictions to df
                    self.cv_scores_[f'{model_name}_{level}'] = cv_scores
                    pred_df = pd.concat(fold_preds)
                    df = df.merge(pred_df, how='left', on='id')
                    metafeatures.append(f'{model_name}_{level}')
                    
                    # Refit model on entire feature columns for inference
                    model.fit(df[feature_cols], df.sentiment)
                    
        # Save learned metafeatures
        self.metafeatures_ = df[metafeatures]
        return self
    
    def predict_proba(self, df):
        """Return classification probabilities."""
        
        # Iterate over layers to make predictions
        df = df.copy()
        for m in range(len(self.model_dict_list)):
            
            # Get models for current layer
            model_dict = self.model_dict_list[m]
            level = m + 1
            
            # Get feature columns to use for prediction
            if m == 0:
                feature_cols = ['review']
            else:
                prev_names = self.model_dict_list[m-1].keys()
                feature_cols = [f"{model_name}_{level-1}" for model_name in prev_names]

            # Append predictions to test DataFrame
            for model_name in model_dict.keys():
                model = model_dict[model_name]
                pred = model.predict_proba(df[feature_cols])[:, 1] 
                df.loc[:, f"{model_name}_{level}"] = pred
                    
        # Return last predictions
        return np.c_[1 - pred, pred]

    def _predict_fold(self, df, feature_cols, fold, model_name, model, level):
        "Make out-of-fold predictions. Return predict probas and AUC."
        
        X_train = df[df.kfold != fold][feature_cols]
        y_train = df[df.kfold != fold].sentiment.values
        
        X_valid = df[df.kfold == fold][feature_cols] 
        y_valid = df[df.kfold == fold].sentiment.values
        pred_id = df[df.kfold == fold].id

        # Fit model
        model.fit(X_train, y_train)
        
        # Return fold predictions along with fold AUC
        pred = model.predict_proba(X_valid)[:, 1] 
        auc = metrics.roc_auc_score(y_valid, pred)
        return pd.DataFrame({"id": pred_id, f"{model_name}_{level}": pred}), auc

Define the models that we will use at each level.

In [31]:
# Base models
level1 = {
    'lr': make_pipeline(
        ReviewColumnExtractor(),
        TfidfVectorizer(max_features=1000),
        linear_model.LogisticRegression(random_state=42)
    ), 
    
    'lr_cnt': make_pipeline(
        ReviewColumnExtractor(),
        CountVectorizer(), 
        linear_model.LogisticRegression(solver='liblinear', random_state=42)
    ), 
}

# Meta models
level2 = {
    'lr': linear_model.LogisticRegression(),
    'linreg': make_pipeline(StandardScaler(), LinearRegressionClassifier()),
    'xgb': XGBClassifier(eval_metric="logloss", use_label_encoder=False, nthread=1, random_state=42)
}

# Meta models
level3 = {
    'linreg': make_pipeline(StandardScaler(), LinearRegressionClassifier()),
    'xgb': XGBClassifier(eval_metric="logloss", use_label_encoder=False, nthread=1)
}

# Blender head: rank true for linear reg.
level4 = {'blender': Blender(rank=True, random_state=42)}

:::{caution}
Setting `nthread=1` for `XGBClassifier` decreases train time for the parallel stacker from ~250s to ~100s. This goes from worse to better than sequential. 

See also https://github.com/dmlc/xgboost/issues/2163 and other related issues.
:::

Start with timing experiments.

In [32]:
model_dict_list = [level1, level2, level3, level4]

In [33]:
times = []
for i in range(3):
    start_time = time.time()
    stack_parallel = StackingClassifierParallel(model_dict_list, n_jobs=-1)
    stack_parallel.fit(df_train)
    times.append(time.time() - start_time)
    
times = np.array(times)
times.mean(), times.std()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   36.7s finished


  0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished


...

  0%|          | 0/2 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.7s finished


  0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.3s finished


(121.40519380569458, 1.4494119313232572)

In [34]:
times = []
for i in range(3):
    start_time = time.time()
    stack = StackingClassifier(model_dict_list)
    stack.fit(df_train)
    times.append(time.time() - start_time)
    
times = np.array(times)
times.mean(), times.std()


Level 1 preds: lr
fold=0, auc=0.935408483852121
fold=1, auc=0.9348064837016209
fold=2, auc=0.9421081921081921
fold=3, auc=0.9339369339369339
fold=4, auc=0.9282464282464282

Level 1 preds: lr_cnt
fold=0, auc=0.9432962358240589
fold=1, auc=0.9477042369260591
fold=2, auc=0.9484371984371985
fold=3, auc=0.945093695093695
fold=4, auc=0.9360944360944361

Level 2 preds: lr
fold=0, auc=0.947956736989184
fold=1, auc=0.9491119872779968
fold=2, auc=0.9543014543014543
fold=3, auc=0.9484136984136985
fold=4, auc=0.9417046917046917

Level 2 preds: linreg
fold=0, auc=0.9488732372183093
fold=1, auc=0.951178487794622
fold=2, auc=0.9550009550009549
fold=3, auc=0.9496999496999498
fold=4, auc=0.9422389422389422

Level 2 preds: xgb
fold=0, auc=0.9450191112547779
fold=1, auc=0.9460573615143403
fold=2, auc=0.9494060744060744
fold=3, auc=0.9464370714370715
fold=4, auc=0.9387995637995638

Level 3 preds: linreg
fold=0, auc=0.9490357372589344
fold=1, auc=0.950797987699497
fold=2, auc=0.9551532051532052
fold=3, au

(154.09294215838113, 0.8956691414277855)

Observe that parallel training has a ~20% speed up over sequential training!

Testing if predictions agree:

In [35]:
times = []
for i in range(3):
    start_time = time.time()
    parallel_pred = stack_parallel.predict_proba(df_test)[:, 1]
    times.append(time.time() - start_time)
    
times = np.array(times)
times.mean(), times.std()

(2.9814700285593667, 0.04484930409238955)

In [36]:
times = []
for i in range(3):
    start_time = time.time()
    usual_pred = stack.predict_proba(df_test)[:, 1]
    times.append(time.time() - start_time)
    
times = np.array(times)
times.mean(), times.std()

(2.958096981048584, 0.0031837037109286163)

In [37]:
print('parallel AUC:', metrics.roc_auc_score(df_test.sentiment, parallel_pred))
print('usual AUC:   ', metrics.roc_auc_score(df_test.sentiment, usual_pred))

parallel AUC: 0.9481989895576532
usual AUC:    0.9482473100524548


Testing if the results agree at the fold level:

In [38]:
# parallel
pd.DataFrame(stack_parallel.cv_scores_).describe().loc[['mean', 'std']]

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,linreg_3,xgb_3,blender_4
mean,0.934901,0.944123,0.948296,0.949398,0.945122,0.949542,0.944967,0.949534
std,0.004935,0.004935,0.004481,0.004639,0.003481,0.004384,0.004823,0.004386


In [39]:
# sequential
pd.DataFrame(stack.cv_scores_).describe().loc[['mean', 'std']]

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,linreg_3,xgb_3,blender_4
mean,0.934901,0.944125,0.948298,0.949398,0.945144,0.949541,0.944763,0.949533
std,0.004935,0.004938,0.004481,0.004642,0.003903,0.004423,0.004716,0.004418


Checking if the two learned different model weights (previously I forgot to clone the models in `model_dict_list` so the models learned the same weights):

In [40]:
print(stack_parallel.model_dict_list[3]['blender'].coef_)
print(stack.model_dict_list[3]['blender'].coef_)

[ 0.26022385 -0.00150484]
[ 0.25856748 -0.00204217]


In [41]:
stack_parallel.metafeatures_.head()

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,linreg_3,xgb_3,blender_4
0,0.830411,0.999715,0.940972,0.091677,0.957805,0.095576,0.997882,831.638265
1,0.649919,0.999999,0.885638,0.085748,0.903252,0.070906,0.961036,762.919991
2,0.865389,0.952606,0.940872,0.051268,0.759027,0.063793,0.957414,741.906520
3,0.209188,0.005925,0.062271,-0.102864,0.041555,-0.088847,0.049510,286.679626
4,0.815459,0.996735,0.937029,0.070447,0.968837,0.082726,0.916480,799.852847


In [42]:
stack.metafeatures_.head()

,lr_1,lr_cnt_1,lr_2,linreg_2,xgb_2,linreg_3,xgb_3,blender_4
0,0.830411,0.999716,0.940971,0.091679,0.961496,0.095929,0.992587,837.532726
1,0.649919,0.999999,0.885644,0.085754,0.934175,0.072787,0.949495,771.347888
2,0.865389,0.952598,0.940868,0.051267,0.786626,0.063910,0.928105,744.778776
3,0.209188,0.005921,0.062270,-0.102863,0.041536,-0.088853,0.051232,288.942894
4,0.815459,0.996732,0.937028,0.070448,0.956573,0.081564,0.909145,797.931596
